In [1]:
import requests
import json

def get_market_orders_raw(region_id=10000002, page=1, order_type='all'):
    """
    Получает сырой ответ с ордерами рынка для региона.
    
    Args:
        region_id (int): ID региона. The Forge = 10000002.
        page (int): Номер страницы пагинации.
        order_type (str): 'all', 'buy' или 'sell'.
    """
    # Базовый URL эндпоинта
    url = f"https://esi.evetech.net/latest/markets/{region_id}/orders/"
    
    # Параметры запроса
    params = {
        'datasource': 'tranquility',  # Основной игровой сервер
        'order_type': order_type,
        'page': page
    }
    
    try:
        print(f"Запрос данных для региона ID {region_id} (страница {page})...")
        response = requests.get(url, params=params)
        
        # Проверка успешности запроса
        response.raise_for_status()
        
        # Получение сырого JSON
        raw_data = response.json()
        
        # Сохранение сырого ответа в файл для анализа
        filename = f"market_orders_region_{region_id}_page_{page}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(raw_data, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Успешно! Сырые данные сохранены в файл: {filename}")
        print(f"   Получено ордеров: {len(raw_data)}")
        print(f"   Статус ответа: {response.status_code}")
        
        # Вывод заголовков пагинации (если есть)
        if 'x-pages' in response.headers:
            total_pages = int(response.headers['x-pages'])
            print(f"   Всего страниц: {total_pages}")
            if total_pages > 1:
                print(f"   Внимание: Для получения всех ордеров нужно выполнить {total_pages} запросов.")
        
        return raw_data, response.headers
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Ошибка при выполнении запроса: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"   Статус ответа: {e.response.status_code}")
            print(f"   Текст ошибки: {e.response.text[:200]}")
        return None, None




In [2]:
# Получение первой страницы всех ордеров
raw_orders, headers = get_market_orders_raw(region_id=10000002, page=1, order_type='all')

# Пример: как получить доступ к данным
if raw_orders and len(raw_orders) > 0:
    print("\nПример первого ордера в ответе:")
    print(json.dumps(raw_orders[0], indent=2, ensure_ascii=False))

Запрос данных для региона ID 10000002 (страница 1)...
✅ Успешно! Сырые данные сохранены в файл: market_orders_region_10000002_page_1.json
   Получено ордеров: 1000
   Статус ответа: 200
   Всего страниц: 443
   Внимание: Для получения всех ордеров нужно выполнить 443 запросов.

Пример первого ордера в ответе:
{
  "duration": 90,
  "is_buy_order": false,
  "issued": "2025-11-09T09:10:01Z",
  "location_id": 60003760,
  "min_volume": 1,
  "order_id": 7181697027,
  "price": 77000000.0,
  "range": "region",
  "system_id": 30000142,
  "type_id": 14144,
  "volume_remain": 1,
  "volume_total": 1
}
